# Object Detection (Centralized)

In [ ]:
# 최적의 모델을 얻고 싶을 경우, 세션 재시작 후 CPU 코어 8개로 설정.
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pybboxes albumentations==1.3.0 --upgrade typing-extensions torch

In [ ]:
import albumentations as A
from augmentation import get_bboxes_list, apply_aug
import yaml

In [ ]:
import shutil
import zipfile
import numpy as np
import pandas as pd
from PIL import Image
import PIL
import json
import sys
import shutil
from distutils.dir_util import copy_tree
# import imagesize
# import ptitprince as pt
from shutil import copyfile
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
import seaborn as sns
import re
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import random
import torch
from tqdm.notebook import tqdm
import cv2
from collections import defaultdict
import glob

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
seed_everything()

In [ ]:
client = 'PCB' # All dataset
class_name = ['SCRATCH', 'DELAMINATION', 'POP_CORN_1', 'POP_CORN_2'] # BOXING_TXT_1013

## 데이터 증강

In [ ]:
# # 데이터 증강했으면 재증강하지 않는다.
# # YAML 데이터 생성
# data = {
#     "inp_img_pth": f"./{client}/train/images/",
#     "inp_lab_pth": f"./{client}/train/labels/",
#     "out_img_pth": f"./{client}/train/images/",
#     "out_lab_pth": f"./{client}/train/labels/",
#     "transformed_file_name": "AUG",
#     "CLASSES": class_name
# }

# # YAML 파일에 데이터 쓰기
# with open(f"./{client}/train_contants.yaml", 'w') as yaml_file:
#     yaml.dump(data, yaml_file)

# with open(f"./{client}/train_contants.yaml", 'r') as stream:
#     CONSTANTS = yaml.safe_load(stream)

# imgs = os.listdir(CONSTANTS["inp_img_pth"])
# for img_file in tqdm(imgs):
#     file_name = img_file[:-4]
#     if 'ipynb_checkpo' in file_name:
#         continue
#     image = cv2.imread(os.path.join(CONSTANTS["inp_img_pth"], img_file))
#     lab_pth = os.path.join(CONSTANTS["inp_lab_pth"], file_name + '.txt')
#     album_bboxes = get_bboxes_list(lab_pth, CONSTANTS['CLASSES'])
#     print(file_name)
#     # x번의 데이터 증강을 수행하고 각각의 결과를 저장 range(1, x) <- 여기를 변경
#     for i in range(1, 5):
#         aug_file_name = f"{file_name}_AUG_{i}"
#         try:
#             apply_aug(image, album_bboxes, CONSTANTS["out_lab_pth"], CONSTANTS["out_img_pth"], aug_file_name, CONSTANTS['CLASSES'])
#         except Exception as e:
#             print(f"An error occurred for {aug_file_name}: {str(e)}")
#             continue  # 에러 발생 시 해당 부분을 건너뛰고 계속 진행

## YAML 파일 생성

In [ ]:
# yaml 파일 생성
data = { 'train' : os.getcwd() + f'/{client}/train/images',
         'val' : os.getcwd() + f'/{client}/valid/images/',
         'test' : os.getcwd() + f'/{client}/test/images',
         'names' : class_name,
         'nc' : len(class_name) }

with open(f'./{client}/PCB.yaml', 'w') as f:
    yaml.dump(data, f)

with open(f'./{client}/PCB.yaml', 'r') as f:
    pcb_yaml = yaml.safe_load(f)
    display(pcb_yaml)

In [ ]:
!cat ./{client}/PCB.yaml

## YOLOv5n Train

In [ ]:
if os.getcwd() != '/home/work/KISTI_PCB2/yolov5':
    os.chdir('./yolov5')
os.getcwd()

In [ ]:
!python train.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" \
    --epochs 300 \
    --patience 15 \
    --batch 32 \
    --imgsz 640 \
    --weights 'yolov5n.pt' \
    --name '{client}_train_results' \
    --exist-ok

## YOLOv5n Test (inference)

In [ ]:
if os.getcwd() != '/home/work/KISTI_PCB2/yolov5':
    os.chdir('./yolov5')
os.getcwd()

In [ ]:
!python val.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" --weights "/home/work/KISTI_PCB2/yolov5/runs/train/{client}_train_results/weights/best.pt" --task 'test' --name '{client}_result_final(central)' --exist-ok

In [ ]:
!python detect.py --source "/home/work/KISTI_PCB2/{client}/test/images" \
    --weights "/home/work/KISTI_PCB2/yolov5/runs/train/{client}_train_results/weights/best.pt" \
    --exist-ok \
    --line-thickness 2 \
    --name '{client}_detect_results(central)'